In [371]:
from load_data import *
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from utils import *
from test_network import test
from load_data import data_augmentation

from keras.wrappers.scikit_learn import KerasClassifier

In [372]:
dataset_train = 'MIT_split_small/train/'
dataset_test = 'MIT_split_small/test/'
IMG_SIZE = 64
BATCH_SIZE = 16
DENSITY = 'mobilenet_base'
OPTIMIZER='RMSprop'

## Load Data and Augment

In [373]:
config = 'default'
train_datagen = data_augmentation(config)
test_datagen = data_augmentation('test') # sempre a test

In [374]:
train_generator = train_datagen.flow_from_directory(
        dataset_train, 
        target_size=(IMG_SIZE, IMG_SIZE),  
        batch_size=BATCH_SIZE,
        classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
        class_mode='categorical')  

validation_generator = test_datagen.flow_from_directory(
        dataset_test,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
        class_mode='categorical')


Found 400 images belonging to 8 classes.
Found 2288 images belonging to 8 classes.


In [375]:
def getXYfromGenerator(generator):
    X, Y = generator.next()
    batch_index = 1

    while batch_index <= generator.batch_index:
        auxX, auxY = generator.next()
        X = np.concatenate((X, auxX))
        Y = np.concatenate((Y, auxY))
        batch_index = batch_index + 1

    return X, Y


X, Y = getXYfromGenerator(train_generator)

### get data

In [360]:
train_generator.n

400

In [376]:
X, Y = getXYfromGenerator(train_generator)

In [333]:
X.shape

(400, 64, 64, 3)

## Create Model

In [377]:
def create(IMG_SIZE=64, optimizer_param='sgd'):
    
    model = Sequential()
    model.add(Reshape( (IMG_SIZE*IMG_SIZE*3,), input_shape=(IMG_SIZE, IMG_SIZE, 3), name='first') )
    model.add( Dense ( units=512, activation='relu', name='second') )
    model.add( Dense ( units=8, activation='softmax') )
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer_param,
                metrics=['accuracy'])    
    
    return model
        

In [379]:
print('Building model...\n')

model = KerasClassifier(build_fn = create)

model_setup = create(IMG_SIZE)
print(model_setup.summary())


Building model...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first (Reshape)              (None, 12288)             0         
_________________________________________________________________
second (Dense)               (None, 512)               6291968   
_________________________________________________________________
dense_22 (Dense)             (None, 8)                 4104      
Total params: 6,296,072
Trainable params: 6,296,072
Non-trainable params: 0
_________________________________________________________________
None


## Grid Search

In [380]:
param_grid = {'epochs': [10, 2]}
#epochs = [2]
#param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2)

In [ ]:
grid_result = grid.fit(X,Y)